In [34]:
import json
import pandas as pd
import requests
import time
import numpy as np
import os

In [35]:
url = "https://api.opendota.com/api/publicMatches?api_key=062963e6-f4c4-4ae4-9a56-a0136f3c2a18"
response = requests.get(url)
def remove_set(a,b):
    return list(set(a)-set(b))

In [36]:
hero_json_file = os.path.join(".","data","heroes.json")
hero_ids = []
with open(hero_json_file) as json_file:
    data = json.load(json_file)
    for i in range(len(data)):
        currDict = data[i]
        hero_ids.append(currDict['id'])
matchups_all = []
for i in hero_ids:
    for j in hero_ids:
        if i != j:
            matchups_all.append(str(i) + "-" + str(j))

In [37]:
X_all = np.array(matchups_all).reshape(-1,1)


In [38]:
selected_games = []
all_games = []

In [39]:
# al_games = []
# for i in range(0,100):
#     matches = requests.get(url).json()
#     print(i)
#     for j in matches:
#         all_games.append(j)
#     time.sleep(3)

In [40]:
# with open('data3.json', 'w') as f:
#     json.dump(all_games, f)

In [41]:
f = open('data.json',)
first=json.load(f)
j = open('data2.json',)
second=json.load(j)
k = open('data3.json',)
third = json.load(k)
all_games = first + second + third
len(third)

10000

In [42]:
len(all_games)

30000

In [43]:
filtered_match = []
winrate_dict = {}
result_list = []
for match in all_games:
    if match["avg_mmr"] and match["game_mode"] == 22 and match["avg_mmr"] > 3000 and match["lobby_type"] == 7:
        result = int(match["radiant_win"])
        for rhero in list(match["radiant_team"].split(",")):
            for dhero in list(match["dire_team"].split(",")):
                result_list.append([result,rhero,dhero])

In [44]:
df = pd.DataFrame(result_list,columns=["winner","hero1","hero2"])
df['combo'] = df['hero1'] + "-" + df['hero2']
df.head()

,winner,hero1,hero2,combo
0,1,45,58,45-58
1,1,45,50,45-50
2,1,45,14,45-14
3,1,45,44,45-44
4,1,45,16,45-16


In [45]:
len(df['hero1'].unique())

119

In [46]:
X = df['combo']
y = df["winner"]
print(X.shape,y.shape)

(231500,) (231500,)


In [47]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.utils import to_categorical

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [49]:
X_train_np = X_train.to_numpy().reshape(-1,1)
X_test_np = X_test.to_numpy().reshape(-1,1)
X_np = X.to_numpy()

In [50]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [51]:
ohe = OneHotEncoder()
ohe.fit(X_all)
encoded_X_train = ohe.transform(X_train_np)
encoded_X_test = ohe.transform(X_test_np)


In [52]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 100)
classifier = model.fit(encoded_X_train,y_train_categorical)

In [53]:
model.feature_importances_

array([0.        , 0.        , 0.00031959, ..., 0.        , 0.        ,
       0.        ])

In [54]:
prediction = ohe.transform([["136-16"]])
model.predict_proba(prediction)

[array([[1., 0.]]), array([[0., 1.]])]

In [60]:
import joblib
joblib.dump(classifier,"dota_picks_trained.h5")

['dota_picks_trained.h5']

In [56]:
# our_heroes = [9,18,53]
# their_heroes = [10,51,38]
# matchups = []

# check_heroes = remove_set(hero_ids,our_heroes + their_heroes)
# checked_dict = {}
# for i in check_heroes:
#     print(f'checking {i}')
#     for j in their_heroes:
#         sum = 
#         current_pred = model.predict_proba(ohe.
#         current_dict = {j:current_pred[0][0][0]}

In [57]:
# for i in matchups_all:

In [58]:
# current_dict[135]
# def sort_dict_by_value(d, reverse = False):
#   return dict(sorted(d.items(), key = lambda x: x[1], reverse = reverse))

In [59]:
# for i in matchups:
#     print(model.predict_proba(ohe.transform([[i]])))